In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Embeddingの基本


In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [46]:
embedding_sample = embeddings.embed_query("test")
embedding_sample[:5]

[-0.0016985070060851744,
 -0.01390356855942869,
 0.0016152468895438766,
 -0.018369816955919344,
 -0.007179653753743353]

embeddingすると、vectorに変換される。

In [4]:
len(embedding_sample)

1536

In [5]:
embeddings.model

'text-embedding-ada-002'

OpenAIのembeddingでは `text-embedding-ada-002` というモデルが使われており、1536次元に変換される。

複数の文字列を同時に変換することも可能。

In [6]:
multiple_embeddings = embeddings.embed_documents(["Apple", "Orange"])
multiple_embeddings

[[0.014477049403261348,
  -0.003934278727982005,
  -0.002658805418212305,
  -0.03788412467717296,
  -0.005858616600728887,
  0.019037936412374574,
  -0.0177778718350305,
  -0.01692869747581054,
  0.02017473485560401,
  -0.015079688700284245,
  0.004875903153449241,
  0.023571430429838655,
  0.007313854730173861,
  0.004427347455392238,
  -0.021599155520197727,
  -0.004502677483935424,
  0.026379183668536275,
  0.016230183173676953,
  0.016367147200600056,
  -0.029638917078493015,
  -0.027803604333129988,
  0.0037322573936301178,
  0.014004525186757322,
  -0.010388686610652153,
  0.004745787607269072,
  -0.0069851420900132735,
  -8.544172543515129e-05,
  0.0006488649122231573,
  0.014367478367987387,
  -0.011840500266895008,
  0.025050637077730652,
  -0.005194343305326077,
  -0.027228358027756236,
  -0.003793890926348734,
  -0.024584960876308258,
  -0.017942227922280146,
  -0.010963933847348508,
  -0.015559061863192506,
  0.014353782337824117,
  -0.00942994214748083,
  0.010361294550325

In [7]:
list(map(len, multiple_embeddings))

[1536, 1536]

## Vector Databaseの基本

Vector Databaseの一つであるChromaを使ってみる。

### Vector Database用のデータを用意
ChatGPTが処理できる文書の長さに限界があるため、短めに文書を分割しておく。

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index=True   # 分割された位置を保存できる
)

In [9]:
from langchain.document_loaders import ArxivLoader
arxiv_no = "2311.00681"
arxiv_loader = ArxivLoader(arxiv_no)
arxiv_doc = arxiv_loader.load_and_split(text_splitter)

In [10]:
len(arxiv_doc)

34

一つの論文が34個のDocumentに分けられました。

それぞれのdocumentにmetadataが付与されており、このmetadataも一緒にvector databaseに保存されます。

In [11]:
arxiv_doc[0].metadata

{'Published': '2023-11-01',
 'Title': 'Are Large Language Models Reliable Judges? A Study on the Factuality Evaluation Capabilities of LLMs',
 'Authors': 'Xue-Yong Fu, Md Tahmid Rahman Laskar, Cheng Chen, Shashi Bhushan TN',
 'Summary': "In recent years, Large Language Models (LLMs) have gained immense attention\ndue to their notable emergent capabilities, surpassing those seen in earlier\nlanguage models. A particularly intriguing application of LLMs is their role as\nevaluators for texts produced by various generative models.\n  In this study, we delve into the potential of LLMs as reliable assessors of\nfactual consistency in summaries generated by text-generation models.\nInitially, we introduce an innovative approach for factuality assessment using\nLLMs. This entails employing a singular LLM for the entirety of the\nquestion-answering-based factuality scoring process. Following this, we examine\nthe efficacy of various LLMs in direct factuality scoring, benchmarking them\nagainst

### Vector Databaseに保存

In [12]:
persist_directory = "./chroma_db"

In [13]:
!rm -rf $persist_directory

In [14]:
from langchain.vectorstores import Chroma

collection_name = "chroma_test"
db = Chroma.from_documents(arxiv_doc, embeddings, collection_name=collection_name, persist_directory=persist_directory)

In [15]:
!ls $persist_directory

chroma.sqlite3                       e8ea4e0d-fa4c-4b03-b948-c744059b6fb8


persist_directoryに指定したディレクトリにファイルができていることが確認できる。

オブジェクトの数は、　`_collection.count()` を使って取得可能

In [16]:
db._collection.count()

34

34個のオブジェクトが保存されていることがわかる。
これは分割されたDocumentの一緒となる。

getメソッドに引数を与えなければすべてのドキュメントを取得できる

In [17]:
db.get?

Signature:
db.get(
    ids: 'Optional[OneOrMany[ID]]' = None,
    where: 'Optional[Where]' = None,
    limit: 'Optional[int]' = None,
    offset: 'Optional[int]' = None,
    where_document: 'Optional[WhereDocument]' = None,
    include: 'Optional[List[str]]' = None,
) -> 'Dict[str, Any]'
Docstring:
Gets the collection.

Args:
    ids: The ids of the embeddings to get. Optional.
    where: A Where type dict used to filter results by.
           E.g. `{"color" : "red", "price": 4.20}`. Optional.
    limit: The number of documents to return. Optional.
    offset: The offset to start returning results from.
            Useful for paging results with limit. Optional.
    where_document: A WhereDocument type dict used to filter by the documents.
                    E.g. `{$contains: "hello"}`. Optional.
    include: A list of what to include in the results.
             Can contain `"embeddings"`, `"metadatas"`, `"documents"`.
             Ids are always included.
             Defaults to `[

In [18]:
all_docs = db.get()
all_docs

{'ids': ['d662276e-88fe-11ee-8738-66a9796eac2f',
  'd6622818-88fe-11ee-8738-66a9796eac2f',
  'd662282c-88fe-11ee-8738-66a9796eac2f',
  'd6622840-88fe-11ee-8738-66a9796eac2f',
  'd6622854-88fe-11ee-8738-66a9796eac2f',
  'd6622868-88fe-11ee-8738-66a9796eac2f',
  'd662287c-88fe-11ee-8738-66a9796eac2f',
  'd662289a-88fe-11ee-8738-66a9796eac2f',
  'd66228ae-88fe-11ee-8738-66a9796eac2f',
  'd66228c2-88fe-11ee-8738-66a9796eac2f',
  'd66228d6-88fe-11ee-8738-66a9796eac2f',
  'd66228e0-88fe-11ee-8738-66a9796eac2f',
  'd66228f4-88fe-11ee-8738-66a9796eac2f',
  'd6622908-88fe-11ee-8738-66a9796eac2f',
  'd6622912-88fe-11ee-8738-66a9796eac2f',
  'd6622926-88fe-11ee-8738-66a9796eac2f',
  'd6622930-88fe-11ee-8738-66a9796eac2f',
  'd6622944-88fe-11ee-8738-66a9796eac2f',
  'd662294e-88fe-11ee-8738-66a9796eac2f',
  'd6622962-88fe-11ee-8738-66a9796eac2f',
  'd6622976-88fe-11ee-8738-66a9796eac2f',
  'd6622980-88fe-11ee-8738-66a9796eac2f',
  'd6622994-88fe-11ee-8738-66a9796eac2f',
  'd662299e-88fe-11ee-8738-

In [19]:
len(all_docs["ids"])

34

idを指定すれば個別のドキュメントも取得可能。

In [20]:
doc_id = all_docs["ids"][0]
sample_doc = db.get(doc_id)
sample_doc

{'ids': ['d662276e-88fe-11ee-8738-66a9796eac2f'],
 'embeddings': None,
 'metadatas': [{'Authors': 'Xue-Yong Fu, Md Tahmid Rahman Laskar, Cheng Chen, Shashi Bhushan TN',
   'Published': '2023-11-01',
   'Summary': "In recent years, Large Language Models (LLMs) have gained immense attention\ndue to their notable emergent capabilities, surpassing those seen in earlier\nlanguage models. A particularly intriguing application of LLMs is their role as\nevaluators for texts produced by various generative models.\n  In this study, we delve into the potential of LLMs as reliable assessors of\nfactual consistency in summaries generated by text-generation models.\nInitially, we introduce an innovative approach for factuality assessment using\nLLMs. This entails employing a singular LLM for the entirety of the\nquestion-answering-based factuality scoring process. Following this, we examine\nthe efficacy of various LLMs in direct factuality scoring, benchmarking them\nagainst traditional measures an

### Vector Databaseを使った検索

`smilarity_search` メソッドを使って検索をすることができる。

In [21]:
db.similarity_search?

Signature:
db.similarity_search(
    query: 'str',
    k: 'int' = 4,
    filter: 'Optional[Dict[str, str]]' = None,
    **kwargs: 'Any',
) -> 'List[Document]'
Docstring:
Run similarity search with Chroma.

Args:
    query (str): Query text to search for.
    k (int): Number of results to return. Defaults to 4.
    filter (Optional[Dict[str, str]]): Filter by metadata. Defaults to None.

Returns:
    List[Document]: List of documents most similar to the query text.
File:      ~/.pyenv/versions/3.11.5/lib/python3.11/site-packages/langchain/vectorstores/chroma.py
Type:      method

In [22]:
search_results = db.similarity_search("LLaMA")
search_results[0]

Document(page_content='progress across a broad spectrum of NLP tasks,\nfrom text classification to generation, language\ntranslation, and beyond (Laskar et al., 2023a,c).\nGiven the capabilities of these LLMs, our re-\nsearch explores the possibility of utilizing LLMs\nfor the critical task of factual consistency evalua-\ntion (Dubois et al., 2023; Liu et al., 2023b; Manakul\net al., 2023; Tang et al., 2022; Laban et al., 2023).\nTo assess the factual consistency of a model, one\ncommon approach is the utilization of a question-\nanswering (QA) pipeline (Huang et al., 2021). Tra-\nditionally, the evaluation of factuality using QA\nsystems has involved the use of separate, distinct\nmodels for each of the following tasks: answer se-\nlection, question generation, and question answer-\ning (Huang et al., 2021). However, this approach\ninvolves the intricate task of coordinating between\nthese disparate models, potentially resulting in in-\nefficiencies in real-world scenarios. Additional

`similarity_search_with_score` メソッドを使えば、ドキュメントとの距離の近さも返してくれる。

In [23]:
db.similarity_search_with_score?

Signature:
db.similarity_search_with_score(
    query: 'str',
    k: 'int' = 4,
    filter: 'Optional[Dict[str, str]]' = None,
    where_document: 'Optional[Dict[str, str]]' = None,
    **kwargs: 'Any',
) -> 'List[Tuple[Document, float]]'
Docstring:
Run similarity search with Chroma with distance.

Args:
    query (str): Query text to search for.
    k (int): Number of results to return. Defaults to 4.
    filter (Optional[Dict[str, str]]): Filter by metadata. Defaults to None.

Returns:
    List[Tuple[Document, float]]: List of documents most similar to
    the query text and cosine distance in float for each.
    Lower score represents more similarity.
File:      ~/.pyenv/versions/3.11.5/lib/python3.11/site-packages/langchain/vectorstores/chroma.py
Type:      method

In [24]:
search_results_with_score = db.similarity_search_with_score("LLaMA")
search_results_with_score[0]

(Document(page_content='progress across a broad spectrum of NLP tasks,\nfrom text classification to generation, language\ntranslation, and beyond (Laskar et al., 2023a,c).\nGiven the capabilities of these LLMs, our re-\nsearch explores the possibility of utilizing LLMs\nfor the critical task of factual consistency evalua-\ntion (Dubois et al., 2023; Liu et al., 2023b; Manakul\net al., 2023; Tang et al., 2022; Laban et al., 2023).\nTo assess the factual consistency of a model, one\ncommon approach is the utilization of a question-\nanswering (QA) pipeline (Huang et al., 2021). Tra-\nditionally, the evaluation of factuality using QA\nsystems has involved the use of separate, distinct\nmodels for each of the following tasks: answer se-\nlection, question generation, and question answer-\ning (Huang et al., 2021). However, this approach\ninvolves the intricate task of coordinating between\nthese disparate models, potentially resulting in in-\nefficiencies in real-world scenarios. Additiona

In [25]:
[x[1] for x in search_results_with_score]

[0.42234497482685956,
 0.4338907856430122,
 0.44076382254582985,
 0.44147816213644003]

小さい値であればあるほど、ドキュメントとクエリの距離が近い、つまり、より関連しているといえる。

## RetrievalQAを使った検索

In [26]:
import langchain

langchain.verbose = True

### Retrieverの作成

ChatGPTとVector Databaseを連携するためにRetrieverを作成する。

Retrieverは `as_retriever` メソッドを使って作成できる。
検索タイプを指定したり、検索結果数や距離のしきい値を決めることができる。

In [27]:
db.as_retriever?

Signature: db.as_retriever(**kwargs: 'Any') -> 'VectorStoreRetriever'
Docstring:
Return VectorStoreRetriever initialized from this VectorStore.

Args:
    search_type (Optional[str]): Defines the type of search that
        the Retriever should perform.
        Can be "similarity" (default), "mmr", or
        "similarity_score_threshold".
    search_kwargs (Optional[Dict]): Keyword arguments to pass to the
        search function. Can include things like:
            k: Amount of documents to return (Default: 4)
            score_threshold: Minimum relevance threshold
                for similarity_score_threshold
            fetch_k: Amount of documents to pass to MMR algorithm (Default: 20)
            lambda_mult: Diversity of results returned by MMR;
                1 for minimum diversity and 0 for maximum. (Default: 0.5)
            filter: Filter by document metadata

Returns:
    VectorStoreRetriever: Retriever class for VectorStore.

Examples:

.. code-block:: python

    # Re

In [28]:
retriever = db.as_retriever()

### RetrievalQAで回答を生成してみる

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-0613")
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [31]:
chain.run("what is LLaMA?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
controlling for confounding variables, potentially
mitigating the risk of inaccurate high correlation
measures (Pagnoni et al., 2021).
3
Methodology
In this section, we present our methods: (i) Using
LLMs as QA-based factuality metric, and (ii) Using
LLMs for direct factuality scoring. Below, we first
present these methods.
(i) QA-based Factuality Metric via LLMs:
The reason we chose to incorporate LLMs into
the QA-based factuality metric is that it is more re-
liable than most other existing automatic factuality
metrics for assessing the factual consistency of a
model (Huang et al., 2021). The typical process of
using QA-based systems as factuality evaluators is

'LLaMA stands for "Llama: Open and efficient foundation language models." It is a language model that has been developed for various natural language processing (NLP) tasks, including text classification, generation, language translation, and more. LLaMA is known for its impressive capabilities and has been utilized in research exploring the evaluation of factual consistency in text generation models.'

ChatGPTに投げるプロンプトの冒頭に
```
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
```
と書いてあり、このメッセージに続く形で、contextを渡していることがわかる。

今回のクエリと近いドキュメントの中身とpromptに組み込まれたcontextの関係を調べてみる。

In [32]:
print(db.similarity_search("what is LLaMA?")[0].page_content)

controlling for confounding variables, potentially
mitigating the risk of inaccurate high correlation
measures (Pagnoni et al., 2021).
3
Methodology
In this section, we present our methods: (i) Using
LLMs as QA-based factuality metric, and (ii) Using
LLMs for direct factuality scoring. Below, we first
present these methods.
(i) QA-based Factuality Metric via LLMs:
The reason we chose to incorporate LLMs into
the QA-based factuality metric is that it is more re-
liable than most other existing automatic factuality
metrics for assessing the factual consistency of a
model (Huang et al., 2021). The typical process of
using QA-based systems as factuality evaluators is
comprised of 3 tasks:
(i) Answer Selection: The commencement of
this procedure involves extracting key points, re-
ferred to as “answers” from the provided summary.
(ii) Question Generation: After identifying the
answers, the next step is to formulate questions
based on these answers, using the summary as the
context.


In [33]:
print(db.similarity_search("what is LLaMA?")[1].page_content)

progress across a broad spectrum of NLP tasks,
from text classification to generation, language
translation, and beyond (Laskar et al., 2023a,c).
Given the capabilities of these LLMs, our re-
search explores the possibility of utilizing LLMs
for the critical task of factual consistency evalua-
tion (Dubois et al., 2023; Liu et al., 2023b; Manakul
et al., 2023; Tang et al., 2022; Laban et al., 2023).
To assess the factual consistency of a model, one
common approach is the utilization of a question-
answering (QA) pipeline (Huang et al., 2021). Tra-
ditionally, the evaluation of factuality using QA
systems has involved the use of separate, distinct
models for each of the following tasks: answer se-
lection, question generation, and question answer-
ing (Huang et al., 2021). However, this approach
involves the intricate task of coordinating between
these disparate models, potentially resulting in in-
efficiencies in real-world scenarios. Additionally,


実際に距離の近いドキュメントを、距離が近い順に並べてpromptの入力として使っている事がわかる。

次に全く関係のない質問を投げてみる。

In [34]:
nvidia_question = "What are NVIDIA's latest revenues?"
chain.run(nvidia_question)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Google. 2023. Palm 2 technical report. Goole AI.
Or Honovich, Roee Aharoni, Jonathan Herzig, Hagai
Taitelbaum, Doron Kukliansy, Vered Cohen, Thomas
Scialom, Idan Szpektor, Avinatan Hassidim, and
Yossi Matias. 2022.
True: Re-evaluating factual
consistency evaluation. In Proceedings of the 2022
Conference of the North American Chapter of the
Association for Computational Linguistics: Human
Language Technologies, pages 3905–3920.
Yichong Huang, Xiachong Feng, Xiaocheng Feng, and
Bing Qin. 2021. The factual inconsistency problem
in abstractive text summarization: A survey. arXiv
preprint arXiv:2104.14839.
Elena Khasanova, Pooja Hiranandani, Shayna Gardiner,
Cheng Che

"I'm sorry, but I don't have access to real-time information. It's best to check the latest financial reports or news articles to find out NVIDIA's latest revenues."

期待した答え `I don't know` とは返ってきていないが、正しい答えを返せないことがわかる。

### 別のドキュメントを追加して回答を作成
全く別のドキュメントを追加して回答を作成してみる。

In [35]:
from langchain.document_loaders import WebBaseLoader

html_loader = WebBaseLoader("https://nvidianews.nvidia.com/news/nvidia-announces-financial-results-for-third-quarter-fiscal-2024")
html_doc = html_loader.load_and_split()

In [36]:
len(html_doc)

13

In [37]:
db = Chroma.from_documents(html_doc, embeddings, collection_name=collection_name, persist_directory=persist_directory)

In [38]:
db._collection.count()

47

ドキュメントの数が増えていることがわかる。

In [39]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
chain.run(nvidia_question)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Professional Visualization

Third-quarter revenue was $416 million, up 10% from the previous quarter and up 108% from a year ago.
Announced that Mercedes-Benz is using NVIDIA Omniverse to create digital twins to help plan, design, build and operate its manufacturing and assembly facilities around the world.
Announced a new line of desktop workstations with NVIDIA RTX™ 6000 Ada Generation GPUs and NVIDIA ConnectX® smart interface cards for training smaller AI models, fine-tuning models and running inference locally.

Automotive 

Third-quarter revenue was $261 million, up 3% from the previous quarter and up 4% from a year ago.
Furthered its collaboration with Foxc

"NVIDIA's latest revenues for the third quarter of fiscal 2024 were $18.12 billion."

### 回答のソースとなるドキュメントを取得
回答のソースとなるドキュメントは `return_source_cocuments=True` とすると取得できる。

In [41]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)
res = chain(nvidia_question)   # return_source_documentsを使うときはrunができない



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Professional Visualization

Third-quarter revenue was $416 million, up 10% from the previous quarter and up 108% from a year ago.
Announced that Mercedes-Benz is using NVIDIA Omniverse to create digital twins to help plan, design, build and operate its manufacturing and assembly facilities around the world.
Announced a new line of desktop workstations with NVIDIA RTX™ 6000 Ada Generation GPUs and NVIDIA ConnectX® smart interface cards for training smaller AI models, fine-tuning models and running inference locally.

Automotive 

Third-quarter revenue was $261 million, up 3% from the previous quarter and up 4% from a year ago.
Furthered its collaboration with Foxc

In [42]:
len(res['source_documents'])

4

retrieverで検索するドキュメントはデフォルトで4なので、4つのドキュメントがソースとして返される。

retrieverが検索するドキュメントの数などはあとから変更することも可能。

In [43]:
retriever.search_kwargs["k"] = 1

In [44]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)
res = chain("How much is Nvidia's latest revenue?")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Diluted earnings per share
$4.02
$2.70
$0.58
Up 49%
Up 593%


 
 
 
 
 
 



Outlook
NVIDIA’s outlook for the fourth quarter of fiscal 2024 is as follows:

Revenue is expected to be $20.00 billion, plus or minus 2%.
GAAP and non-GAAP gross margins are expected to be 74.5% and 75.5%, respectively, plus or minus 50 basis points.
GAAP and non-GAAP operating expenses are expected to be approximately $3.17 billion and $2.20 billion, respectively.
GAAP and non-GAAP other income and expense are expected to be an income of approximately $200 million, excluding gains and losses from non-affiliated investments.
GAAP and non-GAAP tax rates are expected to be 15.0%, plus or 

In [45]:
len(res["source_documents"])

1

ソースドキュメントの数が1になっていることがわかる。